# 6.1 Sets of ordered pairs

In [1]:
# install dependencies
%pip install -q amplpy pandas

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



An ordered pair of objects, whether numbers or strings, is written with the objects
separated by a comma and surrounded by parentheses:
```
("PITT","STL")
("bands",5)
(3,101)
```
As the term "ordered" suggests, it makes a difference which object comes first;
("STL","PITT") is not the same as ("PITT","STL"). The same object may
appear both first and second, as in ("PITT","PITT").

Pairs can be collected into sets, just like single objects. A comma-separated list of
pairs may be enclosed in braces to denote a literal set of ordered pairs:


```
{("PITT","STL"),("PITT","FRE"),("PITT","DET"),("CLEV","FRE")}
{(1,1),(1,2),(1,3),(2,1),(2,2),(2,3),(3,1),(3,2),(3,3)}
```
Because sets of ordered pairs are often large and subject to change, however, they seldom
appear explicitly in AMPL models. Instead they are described symbolically in a variety of
ways.

The set of all ordered pairs from two given sets appears frequently in our examples.
In the transportation model of [Figure 3-1a](./3_2_an_AMPL_model_for_the_transportation_problem.ipynb#fig-3-1a), for instance, the set of all origin-destination
pairs is written as either of
```
{ORIG, DEST}
{i in ORIG, j in DEST}
```
depending on whether the context requires dummy indices i and j. The multiperiod production
model of [Figure 4-4](../04/4_3_a_model_of_production_and_transportation.ipynb#fig-4-4) uses a set of all pairs from a set of strings (representing products)
and a set of numbers (representing weeks):
```
{PROD, 1..T}
{p in PROD, t in 1..T}
```
Various collections of model components, such as the parameter `revenue` and the variable
`Sell`, are indexed over this set. When individual components are referenced in the
model, they must have two subscripts, as in `revenue[p,t]` or `Sell[p,t]`. The
order of the subscripts is always the same as the order of the objects in the pairs; in this
case the first subscript must refer to a string in `PROD`, and the second to a number in `1..T`.

An indexing expression like `{p in PROD, t in 1..T}` is the AMPL transcription of
a phrase like "for all p in P, t = 1, ... , T " from algebraic notation. There is no compelling
reason to think in terms of ordered pairs in this case, and indeed we did not mention
ordered pairs when introducing the multiperiod production model in [Chapter 4](../04/04.md). On
the other hand, we can modify the transportation model of [Figure 3-1a](./3_2_an_AMPL_model_for_the_transportation_problem.ipynb#fig-3-1a) to emphasize the
role of origin-destination pairs as "links" between cities, by defining this set of pairs
explicitly:
```
set LINKS = {ORIG,DEST};
```
The shipment costs and amounts can then be indexed over links:
```
param cost {LINKS} >= 0;
var Trans {LINKS} >= 0;
```
In the objective, the sum of costs over all shipments can be written like this:
```
minimize Total_Cost:
  sum {(i,j) in LINKS} cost[i,j] * Trans[i,j];
```
Notice that when dummy indices run over a set of pairs like `LINKS`, they must be defined
in a pair like `(i,j)`. It would be an error to sum over `{k in LINKS}`. The complete
model is shown in [Figure 6-1](../06/6_1_sets_of_ordered_pairs.ipynb#fig-6-1), and should be compared with [Figure 3-1](./3_2_an_AMPL_model_for_the_transportation_problem.ipynb#fig-3-1a). The specification
of the data could be the same as in [Figure 3-1b](./3_2_an_AMPL_model_for_the_transportation_problem.ipynb#fig-3-1b).

<a id='fig-6-1'><center><b>Figure 6-1:</b> Transportation model with all pairs (transp2.mod).</center></a>

In [2]:
%%writefile transp2.mod

set ORIG;                   # origins
set DEST;                   # destinations
set LINKS = {ORIG,DEST};
param supply {ORIG} >= 0;   # amounts available at origins
param demand {DEST} >= 0;   # amounts required at destinations
	   check: sum {i in ORIG} supply[i] = sum {j in DEST} demand[j];
param cost {LINKS} >= 0;    # shipment costs per unit
var Trans {LINKS} >= 0;     # units to be shipped
minimize Total_Cost:
      sum {(i,j) in LINKS} cost[i,j] * Trans[i,j];
subject to Supply {i in ORIG}:
      sum {j in DEST} Trans[i,j] = supply[i];
subject to Demand {j in DEST}:
      sum {i in ORIG} Trans[i,j] = demand[j];

Writing transp2.mod


In [3]:
# Figure 3-1b, data 
import pandas as pd

df_orig = pd.DataFrame(
    [
        ['GARY', 1400],
        ['CLEV', 2600],
        ['PITT', 2900]
    ],
    columns=['ORIG', 'supply']
).set_index('ORIG')

df_dest = pd.DataFrame(
    [
        ['FRA', 900],
        ['DET', 1200],
        ['LAN', 600],
        ['WIN', 400],
        ['STL', 1700],
        ['FRE', 1100],
        ['LAF', 1000]
    ],
    columns=['DEST', 'demand']
).set_index('DEST')

df_cost = pd.DataFrame(
    [
        ['GARY', 39, 14, 11, 14, 16, 82, 8],
        ['CLEV', 27,  9, 12, 9, 26, 95, 17],
        ['PITT', 24, 14, 17, 13, 28, 99, 20]
    ],
    columns=['ORIG', 'FRA', 'DET', 'LAN', 'WIN', 'STL', 'FRE', 'LAF']
).set_index('ORIG')

display(df_orig)
print()
display(df_dest)
print()
display(df_cost)

,supply
ORIG,
GARY,1400
CLEV,2600
PITT,2900


,demand
DEST,
FRA,900
DET,1200
LAN,600
WIN,400
STL,1700
FRE,1100
LAF,1000


,FRA,DET,LAN,WIN,STL,FRE,LAF
ORIG,,,,,,,
GARY,39,14,11,14,16,82,8
CLEV,27,9,12,9,26,95,17
PITT,24,14,17,13,28,99,20


In [4]:
ampl = AMPL()
ampl.read('transp2.mod')
ampl.set_data(df_orig, 'ORIG')
ampl.set_data(df_dest, 'DEST')
ampl.param['cost'] = df_cost
ampl.solve(solver='highs')
ampl.display('Trans')

HiGHS 1.11.0: optimal solution; objective 196200
10 simplex iterations
0 barrier iterations
Trans [*,*] (tr)
:     CLEV   GARY   PITT    :=
DET   1200      0      0
FRA      0      0    900
FRE      0   1100      0
LAF    400    300    300
LAN    600      0      0
STL      0      0   1700
WIN    400      0      0
;

